# Project 01: Missile Geometry 101
### World Defense Organization (WDO) — Spatial Defense Analysis

**Analyst:** Noah Bustard  
**Course:** CS 4545/5993 — Spatial Data and Mapping  
**Date:** Spring 2026  

---

## Mission Overview

As a **Spatial Defense Analyst** for the World Defense Organization, this notebook analyzes 
incoming non-human threats using spatial geometry. The analysis covers:

1. **Plot the World** — Load and visualize world boundaries with the WDO base
2. **Distance & Bearing** — Compute Haversine distances from threat origins to base
3. **Trajectories** — Generate threat paths as LineString geometries
4. **Intersections & Borders** — Determine which countries each trajectory crosses
5. **Damage Zones** — Create buffer zones around trajectory endpoints

> *"If you can see it, you can trust it."* — WDO Field Manual

---
## Setup & Imports

Load all required libraries and configure paths. We use the WDO toolkit (`wdo_geo.py`) 
for custom spatial math (Haversine, bearing, destination point) and standard geospatial 
libraries (GeoPandas, Shapely, Folium) for data handling and visualization.

In [1]:
# ============================================================
# Standard library imports
# ============================================================
import sys
import json
import math
from pathlib import Path
from math import radians, degrees, sin, cos, asin, atan2, sqrt

# ============================================================
# Third-party geospatial imports
# ============================================================
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
from shapely.geometry import Point, LineString, shape, mapping
from shapely.ops import unary_union

# ============================================================
# Add src to path so we can import the WDO toolkit
# ============================================================
ROOT = Path.cwd()
sys.path.insert(0, str(ROOT / 'src'))

from wdo.wdo_geo import (
    haversine_km,
    initial_bearing_deg,
    destination_point,
    trajectory_points,
    LatLon,
    EARTH_RADIUS_KM
)

print('All imports successful.')
print(f'Working directory: {ROOT}')
print(f'Earth radius used: {EARTH_RADIUS_KM} km')

All imports successful.
Working directory: /home/ubuntu/4545_Spatial_Data/Assignments/Project_01
Earth radius used: 6371.0088 km


---
## Configuration

Define the WDO base location, data paths, and threat-type styling parameters. 
The base is located in **Dallas, Texas** (32.7767N, 96.7970W).

In [2]:
# ============================================================
# WDO Base Location (Dallas, TX)
# ============================================================
BASE_LAT = 32.7767
BASE_LON = -96.7970
BASE_NAME = 'WDO Command Center — Dallas, TX'

# ============================================================
# Data Paths
# ============================================================
DATA_DIR       = ROOT / 'data'
THREATS_PATH   = DATA_DIR / 'threats' / 'threats.json'
COUNTRIES_PATH = DATA_DIR / 'world_borders' / 'world_borders.geojson'
MAPS_DIR       = ROOT / 'maps'
MAPS_DIR.mkdir(exist_ok=True)

# Verify data exists
assert DATA_DIR.exists(), f'Missing data/ folder at {DATA_DIR}'
assert THREATS_PATH.exists(), f'Missing threats file at {THREATS_PATH}'
assert COUNTRIES_PATH.exists(), f'Missing world borders geojson at {COUNTRIES_PATH}'
print('All data paths verified.')

# ============================================================
# Threat-type color scheme and damage radii
# Each threat type has a distinct color and buffer radius
# ============================================================
THREAT_STYLES = {
    'alien':    {'color': '#00FF00', 'marker': 'green',  'buffer_km': 150, 'severity': 'High'},
    'orbital':  {'color': '#9400D3', 'marker': 'purple', 'buffer_km': 300, 'severity': 'Critical'},
    'airborne': {'color': '#FF8C00', 'marker': 'orange', 'buffer_km': 100, 'severity': 'Moderate'},
    'kaiju':    {'color': '#FF0000', 'marker': 'red',    'buffer_km': 200, 'severity': 'Severe'},
}

# Proximity threshold for base defense (km)
BASE_PROXIMITY_KM = 500

print('Configuration complete.')
print(f'Base: {BASE_NAME} ({BASE_LAT}, {BASE_LON})')
print(f'Base proximity threshold: {BASE_PROXIMITY_KM} km')

All data paths verified.
Configuration complete.
Base: WDO Command Center — Dallas, TX (32.7767, -96.797)
Base proximity threshold: 500 km


---
## Data Loading

Load the threat data from JSON and the world country boundaries from GeoJSON. 
We use GeoPandas to load the countries data, which gives us a GeoDataFrame with 
Shapely geometries for spatial operations.

In [3]:
# ============================================================
# Load threat data from JSON
# ============================================================
with open(THREATS_PATH, 'r') as f:
    threats_raw = json.load(f)

print(f'Loaded {len(threats_raw)} threats')
print()

# Display threat summary table
threats_df = pd.DataFrame(threats_raw)
threats_df.columns = ['ID', 'Type', 'Origin Lat', 'Origin Lon', 
                       'Bearing (deg)', 'Speed (km/h)', 'Duration (min)']
print('=== THREAT INTELLIGENCE REPORT ===')
print(threats_df.to_string(index=False))

Loaded 10 threats

=== THREAT INTELLIGENCE REPORT ===
  ID     Type  Origin Lat  Origin Lon  Bearing (deg)  Speed (km/h)  Duration (min)
T001    alien   59.470694  -91.766281         181.67         978.6            74.2
T002    alien   10.001950 -114.105017          18.50         875.0            44.0
T003    alien    7.726207 -108.863780          34.36        1361.1            55.2
T004    kaiju    9.890453 -113.895704          49.14          30.3           506.8
T005    kaiju   18.258994  -69.683290         284.39          77.9           281.6
T006    alien   13.083298  -74.781585         306.17        1075.2            45.9
T007    alien   29.344132 -128.681615          85.30        1578.5            52.7
T008 airborne   45.708386 -125.023540         119.36        1103.2            64.6
T009    alien   58.691178  -87.669082         188.52        1031.5            34.8
T010  orbital   50.838205 -120.710048         122.27        4720.5            35.0


In [4]:
# ============================================================
# Load world country boundaries with GeoPandas
# ============================================================
world = gpd.read_file(COUNTRIES_PATH)

print(f'World GeoDataFrame: {len(world)} countries')
print(f'CRS: {world.crs}')
print(f'Columns: {list(world.columns)}')
print()
print('Sample countries:')
print(world[['COUNTRY', 'ISO']].head(10).to_string(index=False))

World GeoDataFrame: 251 countries
CRS: EPSG:4326
Columns: ['FID', 'COUNTRY', 'ISO', 'COUNTRYAFF', 'AFF_ISO', 'geometry']

Sample countries:
            COUNTRY ISO
        Afghanistan  AF
            Albania  AL
            Algeria  DZ
     American Samoa  AS
            Andorra  AD
             Angola  AO
           Anguilla  AI
         Antarctica  AQ
Antigua and Barbuda  AG
          Argentina  AR


---
## Helper Functions

These reusable functions handle map creation, marker placement, and trajectory visualization 
across all milestones.

In [5]:
# ============================================================
# Map creation and visualization helper functions
# ============================================================

def create_base_map(lat, lon, zoom=3):
    """
    Create a Folium map centered on the WDO base with world boundaries.
    
    Parameters:
        lat (float): Base latitude in degrees
        lon (float): Base longitude in degrees  
        zoom (int): Initial zoom level
    
    Returns:
        folium.Map: Interactive map object
    """
    return folium.Map(
        location=[lat, lon],
        zoom_start=zoom,
        tiles='OpenStreetMap',
        control_scale=True
    )


def add_world_borders(m, world_gdf, name='World Borders'):
    """
    Add world country boundaries as a GeoJSON layer to the map.
    
    Parameters:
        m (folium.Map): Target map
        world_gdf (GeoDataFrame): World boundaries data
        name (str): Layer name for the layer control
    """
    style_function = lambda x: {
        'fillColor': '#D4E6F1',
        'color': '#2C3E50',
        'weight': 0.5,
        'fillOpacity': 0.3
    }
    folium.GeoJson(
        world_gdf.__geo_interface__,
        name=name,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=['COUNTRY'],
            aliases=['Country:'],
            style='font-size: 12px;'
        )
    ).add_to(m)


def add_base_marker(m, lat, lon, name='WDO Base'):
    """
    Add the WDO base location as a prominent marker.
    
    Parameters:
        m (folium.Map): Target map
        lat (float): Base latitude
        lon (float): Base longitude
        name (str): Label for the marker
    """
    folium.Marker(
        location=[lat, lon],
        tooltip=name,
        popup=folium.Popup(
            f'<b>{name}</b><br>Lat: {lat}<br>Lon: {lon}',
            max_width=250
        ),
        icon=folium.Icon(color='blue', icon='star', prefix='fa')
    ).add_to(m)


def add_threat_origins(m, threats, threat_styles):
    """
    Add threat origin markers to the map, color-coded by threat type.
    
    Parameters:
        m (folium.Map): Target map
        threats (list): List of threat dictionaries
        threat_styles (dict): Color/style config per threat type
    """
    for t in threats:
        style = threat_styles.get(t['type'], {'color': 'gray'})
        folium.CircleMarker(
            location=[t['origin_lat'], t['origin_lon']],
            radius=8,
            color=style['color'],
            fill=True,
            fillColor=style['color'],
            fillOpacity=0.7,
            tooltip=(
                f"{t['id']} ({t['type']})<br>"
                f"Dist to base: {t['dist_to_base_km']:,.0f} km<br>"
                f"Bearing: {t['bearing_deg']} deg<br>"
                f"Speed: {t['speed_kmh']:,.0f} km/h"
            ),
            popup=folium.Popup(
                f"<b>{t['id']}</b> — {t['type'].upper()}<br>"
                f"Origin: ({t['origin_lat']:.4f}, {t['origin_lon']:.4f})<br>"
                f"Distance to base: {t['dist_to_base_km']:,.1f} km<br>"
                f"Heading: {t['bearing_deg']} deg<br>"
                f"Speed: {t['speed_kmh']:,.1f} km/h<br>"
                f"Duration: {t['duration_min']:.0f} min",
                max_width=300
            )
        ).add_to(m)


def add_trajectories(m, threats, threat_styles):
    """
    Add threat trajectories as colored polylines to the map.
    Also marks the destination (endpoint) of each trajectory.
    
    Parameters:
        m (folium.Map): Target map
        threats (list): Enriched threat list with trajectory_pts
        threat_styles (dict): Color config per threat type
    """
    for t in threats:
        style = threat_styles.get(t['type'], {'color': 'gray'})
        folium.PolyLine(
            locations=t['trajectory_pts'],
            color=style['color'],
            weight=3,
            opacity=0.8,
            tooltip=(
                f"{t['id']} trajectory ({t['type']})<br>"
                f"Distance: {t['total_distance_km']:,.0f} km<br>"
                f"Bearing: {t['bearing_deg']} deg"
            ),
            dash_array='5 5' if t['type'] == 'kaiju' else None
        ).add_to(m)
        
        # Mark the destination point (endpoint)
        folium.CircleMarker(
            location=[t['dest_lat'], t['dest_lon']],
            radius=6,
            color='black',
            fill=True,
            fillColor=style['color'],
            fillOpacity=0.9,
            tooltip=(
                f"{t['id']} endpoint<br>"
                f"({t['dest_lat']:.4f}, {t['dest_lon']:.4f})"
            )
        ).add_to(m)

print('Helper functions defined.')

Helper functions defined.


---
# Milestone 1 — Plot the World

**Goal:** Prove we can load and visualize spatial data.

### Tasks
- Load world country boundaries (GeoJSON)
- Display on an interactive Folium map
- Add the WDO base location as a prominent marker

### Checkpoint Questions
- Can we clearly identify the base on the map? **Yes — blue star marker with popup.**
- Does zooming and panning work correctly? **Yes — interactive Folium map.**

In [6]:
# ============================================================
# MILESTONE 1: Create the base world map with Folium
# ============================================================

m1 = create_base_map(BASE_LAT, BASE_LON, zoom=3)
add_world_borders(m1, world)
add_base_marker(m1, BASE_LAT, BASE_LON, BASE_NAME)
folium.LayerControl().add_to(m1)

# Save the map
m1_path = MAPS_DIR / 'milestone_1_world_map.html'
m1.save(str(m1_path))
print(f'Milestone 1 map saved to: {m1_path}')

m1

Milestone 1 map saved to: /home/ubuntu/4545_Spatial_Data/Assignments/Project_01/maps/milestone_1_world_map.html


---
# Milestone 2 — Distance & Bearing

**Goal:** Reason about spatial relationships numerically and visually.

### Tasks
- Compute the Haversine distance from each threat origin to the WDO base
- Compute the initial bearing from each threat origin toward the base
- Identify the **closest threat**
- Display threat origins as styled markers on the map

### Concepts Reinforced
- Haversine (great-circle) distance
- Initial bearing (forward azimuth)
- Units: degrees vs. kilometers

### Haversine Distance & Initial Bearing — Implementation Verification

Before running the full analysis, we verify our spatial math functions with a known 
test case: **Dallas to Houston** should be approximately 362 km.

In [7]:
# ============================================================
# Sanity check: Dallas -> Houston
# Expected: ~362 km, bearing ~160 deg (south-southeast)
# ============================================================
dallas = (32.7767, -96.7970)
houston = (29.7604, -95.3698)

test_dist = haversine_km(*dallas, *houston)
test_bearing = initial_bearing_deg(*dallas, *houston)

print(f'Dallas -> Houston:')
print(f'  Distance:  {test_dist:.2f} km')
print(f'  Bearing:   {test_bearing:.2f} deg')
print()

# Verify destination point: travel 100 km south from Dallas
dest = destination_point(32.7767, -96.7970, 180, 100)
print(f'100 km due south of Dallas: ({dest.lat:.4f}, {dest.lon:.4f})')
print(f'  (Should be ~31.88 N, same longitude)')
print()

# Verify: same point -> 0 km
zero_dist = haversine_km(32.7767, -96.7970, 32.7767, -96.7970)
print(f'Same point distance: {zero_dist:.6f} km (should be ~0)')

Dallas -> Houston:
  Distance:  361.78 km
  Bearing:   157.61 deg

100 km due south of Dallas: (31.8774, -96.7970)
  (Should be ~31.88 N, same longitude)

Same point distance: 0.000000 km (should be ~0)


In [8]:
# ============================================================
# MILESTONE 2: Compute distances and bearings for all threats
# ============================================================

threats = []
for t in threats_raw:
    # Compute Haversine distance from threat origin to WDO base
    dist_to_base = haversine_km(
        t['origin_lat'], t['origin_lon'],
        BASE_LAT, BASE_LON
    )
    
    # Compute initial bearing from threat origin toward the base
    bearing_to_base = initial_bearing_deg(
        t['origin_lat'], t['origin_lon'],
        BASE_LAT, BASE_LON
    )
    
    threat = {
        'id': t['id'],
        'type': t['type'],
        'origin_lat': t['origin_lat'],
        'origin_lon': t['origin_lon'],
        'bearing_deg': t['bearing_deg'],
        'speed_kmh': t['speed_kmh'],
        'duration_min': t['duration_min'],
        'dist_to_base_km': round(dist_to_base, 2),
        'bearing_to_base_deg': round(bearing_to_base, 2),
    }
    threats.append(threat)

# Display results
dist_df = pd.DataFrame([
    {
        'Threat ID': t['id'],
        'Type': t['type'],
        'Origin': f"({t['origin_lat']:.2f}, {t['origin_lon']:.2f})",
        'Dist to Base (km)': f"{t['dist_to_base_km']:,.1f}",
        'Bearing to Base': f"{t['bearing_to_base_deg']:.1f} deg",
        'Threat Bearing': f"{t['bearing_deg']:.1f} deg",
        'Speed (km/h)': f"{t['speed_kmh']:,.1f}"
    }
    for t in threats
])

print('=== DISTANCE & BEARING ANALYSIS ===')
print(dist_df.to_string(index=False))

=== DISTANCE & BEARING ANALYSIS ===
Threat ID     Type           Origin Dist to Base (km) Bearing to Base Threat Bearing Speed (km/h)
     T001    alien  (59.47, -91.77)           2,991.5       189.4 deg      181.7 deg        978.6
     T002    alien (10.00, -114.11)           3,091.8        32.4 deg       18.5 deg        875.0
     T003    alien  (7.73, -108.86)           3,050.8        22.4 deg       34.4 deg      1,361.1
     T004    kaiju  (9.89, -113.90)           3,090.0        32.0 deg       49.1 deg         30.3
     T005    kaiju  (18.26, -69.68)           3,148.9       306.1 deg      284.4 deg         77.9
     T006    alien  (13.08, -74.78)           3,129.1       318.1 deg      306.2 deg      1,075.2
     T007    alien (29.34, -128.68)           3,049.4        74.6 deg       85.3 deg      1,578.5
     T008 airborne (45.71, -125.02)           2,802.2       111.0 deg      119.4 deg      1,103.2
     T009    alien  (58.69, -87.67)           2,961.2       197.3 deg      188.5 d

In [9]:
# ============================================================
# Identify the closest and farthest threats
# ============================================================
closest = min(threats, key=lambda t: t['dist_to_base_km'])
farthest = max(threats, key=lambda t: t['dist_to_base_km'])

print('=== CLOSEST THREAT ===')
print(f"  ID:       {closest['id']}")
print(f"  Type:     {closest['type']}")
print(f"  Origin:   ({closest['origin_lat']:.4f}, {closest['origin_lon']:.4f})")
print(f"  Distance: {closest['dist_to_base_km']:,.2f} km")
print(f"  Bearing:  {closest['bearing_deg']} deg (threat heading)")
print()
print('=== FARTHEST THREAT ===')
print(f"  ID:       {farthest['id']}")
print(f"  Type:     {farthest['type']}")
print(f"  Distance: {farthest['dist_to_base_km']:,.2f} km")

=== CLOSEST THREAT ===
  ID:       T010
  Type:     orbital
  Origin:   (50.8382, -120.7100)
  Distance: 2,797.33 km
  Bearing:  122.27 deg (threat heading)

=== FARTHEST THREAT ===
  ID:       T005
  Type:     kaiju
  Distance: 3,148.89 km


In [10]:
# ============================================================
# MILESTONE 2: Visualize threat origins on the map
# ============================================================

m2 = create_base_map(BASE_LAT, BASE_LON, zoom=3)
add_world_borders(m2, world)
add_base_marker(m2, BASE_LAT, BASE_LON, BASE_NAME)
add_threat_origins(m2, threats, THREAT_STYLES)

# Add a legend
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
            background-color: white; padding: 10px; border: 2px solid grey;
            border-radius: 5px; font-size: 13px;">
    <b>Threat Types</b><br>
    <i style="background:#00FF00;width:12px;height:12px;display:inline-block;border-radius:50%;"></i> Alien<br>
    <i style="background:#9400D3;width:12px;height:12px;display:inline-block;border-radius:50%;"></i> Orbital<br>
    <i style="background:#FF8C00;width:12px;height:12px;display:inline-block;border-radius:50%;"></i> Airborne<br>
    <i style="background:#FF0000;width:12px;height:12px;display:inline-block;border-radius:50%;"></i> Kaiju<br>
    <i style="background:#3388ff;width:12px;height:12px;display:inline-block;"></i> WDO Base
</div>
'''
m2.get_root().html.add_child(folium.Element(legend_html))
folium.LayerControl().add_to(m2)

m2_path = MAPS_DIR / 'milestone_2_threats.html'
m2.save(str(m2_path))
print(f'Milestone 2 map saved to: {m2_path}')

m2

Milestone 2 map saved to: /home/ubuntu/4545_Spatial_Data/Assignments/Project_01/maps/milestone_2_threats.html


---
# Milestone 3 — Trajectories (Point -> Line)

**Goal:** Turn motion into geometry.

### Tasks
- For each threat, compute a **destination point** after its full duration
- Generate intermediate points along the trajectory
- Construct `LineString` geometries from the point sequences
- Plot all trajectories on the map, color-coded by threat type

### Visual Expectation
We should clearly see where threats started, where they are headed, 
and how paths differ by bearing and speed.

In [11]:
# ============================================================
# MILESTONE 3: Generate trajectories for each threat
# ============================================================

for t in threats:
    # Compute trajectory points using WDO toolkit
    # trajectory_points uses destination_point() internally to calculate
    # intermediate positions along the bearing at regular time intervals
    t['trajectory_pts'] = trajectory_points(
        origin_lat=t['origin_lat'],
        origin_lon=t['origin_lon'],
        bearing_deg=t['bearing_deg'],
        speed_kmh=t['speed_kmh'],
        duration_min=t['duration_min'],
        step_min=2.0
    )
    
    # Total distance: speed * time
    hours = t['duration_min'] / 60.0
    t['total_distance_km'] = round(t['speed_kmh'] * hours, 2)
    
    # Destination point (final position)
    dest = destination_point(
        t['origin_lat'], t['origin_lon'],
        t['bearing_deg'], t['total_distance_km']
    )
    t['dest_lat'] = dest.lat
    t['dest_lon'] = dest.lon
    
    # Create Shapely LineString (note: Shapely uses lon, lat order)
    line_coords = [(lon, lat) for lat, lon in t['trajectory_pts']]
    t['trajectory_line'] = LineString(line_coords)
    t['num_points'] = len(t['trajectory_pts'])

# Display trajectory summary
traj_df = pd.DataFrame([
    {
        'ID': t['id'],
        'Type': t['type'],
        'Origin': f"({t['origin_lat']:.2f}, {t['origin_lon']:.2f})",
        'Destination': f"({t['dest_lat']:.2f}, {t['dest_lon']:.2f})",
        'Distance (km)': f"{t['total_distance_km']:,.1f}",
        'Bearing': f"{t['bearing_deg']:.1f} deg",
        'Points': t['num_points']
    }
    for t in threats
])

print('=== TRAJECTORY ANALYSIS ===')
print(traj_df.to_string(index=False))

=== TRAJECTORY ANALYSIS ===
  ID     Type           Origin      Destination Distance (km)   Bearing  Points
T001    alien  (59.47, -91.77)  (48.59, -92.24)       1,210.2 181.7 deg      38
T002    alien (10.00, -114.11) (15.47, -112.21)         641.7  18.5 deg      23
T003    alien  (7.73, -108.86) (16.95, -102.25)       1,252.2  34.4 deg      28
T004    kaiju  (9.89, -113.90) (11.39, -112.12)         255.9  49.1 deg     254
T005    kaiju  (18.26, -69.68)  (19.05, -73.05)         365.6 284.4 deg     141
T006    alien  (13.08, -74.78)  (17.37, -81.03)         822.5 306.2 deg      23
T007    alien (29.34, -128.68) (29.60, -114.35)       1,386.5  85.3 deg      27
T008 airborne (45.71, -125.02) (39.79, -112.89)       1,187.8 119.4 deg      33
T009    alien  (58.69, -87.67)  (53.36, -89.00)         598.3 188.5 deg      18
T010  orbital (50.84, -120.71)  (34.25, -95.34)       2,753.6 122.3 deg      18


In [12]:
# ============================================================
# MILESTONE 3: Visualize trajectories on the map
# ============================================================

m3 = create_base_map(BASE_LAT, BASE_LON, zoom=3)
add_world_borders(m3, world)
add_base_marker(m3, BASE_LAT, BASE_LON, BASE_NAME)
add_threat_origins(m3, threats, THREAT_STYLES)
add_trajectories(m3, threats, THREAT_STYLES)

# Add legend
legend_html_traj = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
            background-color: white; padding: 10px; border: 2px solid grey;
            border-radius: 5px; font-size: 13px;">
    <b>Threat Trajectories</b><br>
    <span style="color:#00FF00;">&#9644;</span> Alien<br>
    <span style="color:#9400D3;">&#9644;</span> Orbital<br>
    <span style="color:#FF8C00;">&#9644;</span> Airborne<br>
    <span style="color:#FF0000;">- - -</span> Kaiju<br>
    <i style="background:#3388ff;width:12px;height:12px;display:inline-block;"></i> WDO Base<br>
    &#9679; Origin &nbsp; &#9673; Endpoint
</div>
'''
m3.get_root().html.add_child(folium.Element(legend_html_traj))
folium.LayerControl().add_to(m3)

m3_path = MAPS_DIR / 'milestone_3_trajectories.html'
m3.save(str(m3_path))
print(f'Milestone 3 map saved to: {m3_path}')

m3

Milestone 3 map saved to: /home/ubuntu/4545_Spatial_Data/Assignments/Project_01/maps/milestone_3_trajectories.html


---
# Milestone 4 — Intersections & Borders

**Goal:** Determine what the threats interact with.

### Tasks
- Determine which country polygons each trajectory **intersects**
- Check whether any trajectory passes **within 500 km** of the WDO base
- Highlight intersected countries on the map

### Spatial Relationships Used
- `intersects` — does the trajectory cross a country boundary?
- `distance` — how close does the trajectory come to the base?

In [13]:
# ============================================================
# MILESTONE 4: Spatial intersection analysis
# ============================================================

# Build a GeoDataFrame of trajectories
traj_gdf = gpd.GeoDataFrame(
    [{'id': t['id'], 'type': t['type'], 'geometry': t['trajectory_line']} for t in threats],
    crs='EPSG:4326'
)

# Ensure world GeoDataFrame has the same CRS
world_4326 = world.to_crs('EPSG:4326')

print('=== COUNTRY INTERSECTION ANALYSIS ===')
print()

# For each threat, find which countries its trajectory intersects
all_intersected_countries = set()

for t in threats:
    line = t['trajectory_line']
    intersected = world_4326[world_4326.geometry.intersects(line)]
    country_names = list(intersected['COUNTRY'].values)
    
    t['intersected_countries'] = country_names
    all_intersected_countries.update(country_names)
    
    print(f"{t['id']} ({t['type']:8s}): {', '.join(country_names) if country_names else 'No land intersection'}")

print()
print(f'Total unique countries affected: {len(all_intersected_countries)}')
print(f'Countries: {sorted(all_intersected_countries)}')

=== COUNTRY INTERSECTION ANALYSIS ===

T001 (alien   ): Canada
T002 (alien   ): No land intersection
T003 (alien   ): No land intersection
T004 (kaiju   ): No land intersection
T005 (kaiju   ): Dominican Republic, Haiti
T006 (alien   ): No land intersection
T007 (alien   ): Mexico
T008 (airborne): United States
T009 (alien   ): Canada
T010 (orbital ): Canada, United States

Total unique countries affected: 5
Countries: ['Canada', 'Dominican Republic', 'Haiti', 'Mexico', 'United States']


In [14]:
# ============================================================
# Check proximity to WDO base
# Which trajectories pass within the defense threshold?
# ============================================================

print(f'=== BASE PROXIMITY ANALYSIS (threshold: {BASE_PROXIMITY_KM} km) ===')
print()

for t in threats:
    # Find the closest point on the trajectory to the base
    # using Haversine distance for accuracy
    min_dist = float('inf')
    
    for lat, lon in t['trajectory_pts']:
        d = haversine_km(lat, lon, BASE_LAT, BASE_LON)
        if d < min_dist:
            min_dist = d
    
    t['min_dist_to_base_km'] = round(min_dist, 2)
    t['passes_near_base'] = min_dist <= BASE_PROXIMITY_KM
    
    status = 'ALERT — WITHIN RANGE' if t['passes_near_base'] else 'Outside threshold'
    print(f"{t['id']} ({t['type']:8s}): closest approach = {min_dist:>8,.1f} km — {status}")

near_threats = [t for t in threats if t['passes_near_base']]
print()
print(f'Threats within {BASE_PROXIMITY_KM} km of base: {len(near_threats)}')
for t in near_threats:
    print(f"  {t['id']} ({t['type']}): {t['min_dist_to_base_km']:,.1f} km")

=== BASE PROXIMITY ANALYSIS (threshold: 500 km) ===

T001 (alien   ): closest approach =  1,802.2 km — Outside threshold
T002 (alien   ): closest approach =  2,473.4 km — Outside threshold
T003 (alien   ): closest approach =  1,868.2 km — Outside threshold
T004 (kaiju   ): closest approach =  2,846.7 km — Outside threshold
T005 (kaiju   ): closest approach =  2,814.1 km — Outside threshold
T006 (alien   ): closest approach =  2,362.7 km — Outside threshold
T007 (alien   ): closest approach =  1,722.3 km — Outside threshold
T008 (airborne): closest approach =  1,646.6 km — Outside threshold
T009 (alien   ): closest approach =  2,385.1 km — Outside threshold
T010 (orbital ): closest approach =    239.0 km — ALERT — WITHIN RANGE

Threats within 500 km of base: 1
  T010 (orbital): 238.9 km


In [15]:
# ============================================================
# MILESTONE 4: Visualize intersected countries
# ============================================================

m4 = create_base_map(BASE_LAT, BASE_LON, zoom=3)

# Highlight intersected countries in red
intersected_gdf = world_4326[world_4326['COUNTRY'].isin(all_intersected_countries)]
non_intersected_gdf = world_4326[~world_4326['COUNTRY'].isin(all_intersected_countries)]

# Non-intersected countries (light gray)
folium.GeoJson(
    non_intersected_gdf.__geo_interface__,
    name='Unaffected Countries',
    style_function=lambda x: {
        'fillColor': '#D5D8DC',
        'color': '#2C3E50',
        'weight': 0.5,
        'fillOpacity': 0.3
    },
    tooltip=folium.GeoJsonTooltip(fields=['COUNTRY'], aliases=['Country:'])
).add_to(m4)

# Intersected countries (highlighted in red)
folium.GeoJson(
    intersected_gdf.__geo_interface__,
    name='Affected Countries',
    style_function=lambda x: {
        'fillColor': '#E74C3C',
        'color': '#C0392B',
        'weight': 1.5,
        'fillOpacity': 0.45
    },
    tooltip=folium.GeoJsonTooltip(fields=['COUNTRY'], aliases=['Affected Country:'])
).add_to(m4)

# Add base, origins, trajectories
add_base_marker(m4, BASE_LAT, BASE_LON, BASE_NAME)
add_threat_origins(m4, threats, THREAT_STYLES)
add_trajectories(m4, threats, THREAT_STYLES)

# Add base proximity circle (500 km)
folium.Circle(
    location=[BASE_LAT, BASE_LON],
    radius=BASE_PROXIMITY_KM * 1000,
    color='blue',
    fill=True,
    fillColor='blue',
    fillOpacity=0.05,
    weight=2,
    dash_array='10 5',
    tooltip=f'Base defense perimeter ({BASE_PROXIMITY_KM} km)'
).add_to(m4)

# Legend
legend_html_m4 = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
            background-color: white; padding: 10px; border: 2px solid grey;
            border-radius: 5px; font-size: 13px;">
    <b>Intersection Analysis</b><br>
    <i style="background:#E74C3C;width:12px;height:12px;display:inline-block;"></i> Affected Country<br>
    <i style="background:#D5D8DC;width:12px;height:12px;display:inline-block;"></i> Unaffected Country<br>
    <span style="color:blue;">- - -</span> Base Perimeter (500 km)<br>
    <span style="color:#00FF00;">&#9644;</span> Alien &nbsp;
    <span style="color:#9400D3;">&#9644;</span> Orbital<br>
    <span style="color:#FF8C00;">&#9644;</span> Airborne &nbsp;
    <span style="color:#FF0000;">- -</span> Kaiju
</div>
'''
m4.get_root().html.add_child(folium.Element(legend_html_m4))
folium.LayerControl().add_to(m4)

m4_path = MAPS_DIR / 'milestone_4_intersections.html'
m4.save(str(m4_path))
print(f'Milestone 4 map saved to: {m4_path}')

m4

Milestone 4 map saved to: /home/ubuntu/4545_Spatial_Data/Assignments/Project_01/maps/milestone_4_intersections.html


---
# Milestone 5 — Damage Zones (The Bridge)

**Goal:** Prepare data for the next project.

### Tasks
- Create a **buffer zone** around each trajectory endpoint
- Buffer size depends on `threat_type` (see configuration above)
- **Critical:** Buffering must use a projected CRS (meters), not lat/lon degrees
- Determine which countries fall within each damage zone
- Produce a summary table of affected countries with severity ratings

### Buffer Radii by Threat Type
| Threat Type | Buffer Radius | Severity |
|------------|---------------|----------|
| Alien      | 150 km        | High     |
| Orbital    | 300 km        | Critical |
| Airborne   | 100 km        | Moderate |
| Kaiju      | 200 km        | Severe   |

In [16]:
# ============================================================
# MILESTONE 5: Create damage zone buffers
# ============================================================
# CRITICAL: We must project to a meter-based CRS before buffering.
# Using EPSG:3857 (Web Mercator) for meter-based operations.
# Buffering in lat/lon degrees would NOT produce real distance buffers.
# ============================================================

# Create GeoDataFrame of threat endpoints
endpoint_data = []
for t in threats:
    style = THREAT_STYLES[t['type']]
    endpoint_data.append({
        'id': t['id'],
        'type': t['type'],
        'buffer_km': style['buffer_km'],
        'severity': style['severity'],
        'color': style['color'],
        'geometry': Point(t['dest_lon'], t['dest_lat'])  # Shapely: (lon, lat)
    })

endpoints_gdf = gpd.GeoDataFrame(endpoint_data, crs='EPSG:4326')

# Project to EPSG:3857 (meters) for buffering
endpoints_projected = endpoints_gdf.to_crs('EPSG:3857')

# Create buffers in meters (km * 1000)
buffer_geoms = endpoints_projected.apply(
    lambda row: row.geometry.buffer(row['buffer_km'] * 1000),
    axis=1
)

# Create a new GeoDataFrame with buffer geometries and PROPER CRS
# (set_geometry on a new column loses CRS, so we construct a new GDF)
buffers_projected = gpd.GeoDataFrame(
    endpoints_projected.drop(columns='geometry'),
    geometry=buffer_geoms.values,
    crs='EPSG:3857'
)

# Project back to EPSG:4326 for map display and spatial joins
buffers_4326 = buffers_projected.to_crs('EPSG:4326')

print('Damage zone buffers created successfully.')
print(f'CRS verification: {buffers_4326.crs}')
print()
for _, row in buffers_4326.iterrows():
    print(f"{row['id']} ({row['type']}): {row['buffer_km']} km buffer — {row['severity']} severity")

Damage zone buffers created successfully.
CRS verification: EPSG:4326

T001 (alien): 150 km buffer — High severity
T002 (alien): 150 km buffer — High severity
T003 (alien): 150 km buffer — High severity
T004 (kaiju): 200 km buffer — Severe severity
T005 (kaiju): 200 km buffer — Severe severity
T006 (alien): 150 km buffer — High severity
T007 (alien): 150 km buffer — High severity
T008 (airborne): 100 km buffer — Moderate severity
T009 (alien): 150 km buffer — High severity
T010 (orbital): 300 km buffer — Critical severity


In [17]:
# ============================================================
# Determine which countries fall within each damage zone
# ============================================================

print('=== DAMAGE ZONE COUNTRY ANALYSIS ===')
print()

damage_records = []

for idx, buf_row in buffers_4326.iterrows():
    buffer_geom = buf_row.geometry
    
    # Find countries that intersect this damage zone
    affected = world_4326[world_4326.geometry.intersects(buffer_geom)]
    
    t_id = buf_row['id']
    t_type = buf_row['type']
    severity = buf_row['severity']
    
    # Store the affected countries for this threat
    for t in threats:
        if t['id'] == t_id:
            t['damage_zone_countries'] = list(affected['COUNTRY'].values)
            break
    
    if len(affected) > 0:
        for _, country_row in affected.iterrows():
            damage_records.append({
                'country': country_row['COUNTRY'],
                'threat_id': t_id,
                'threat_type': t_type,
                'severity': severity,
                'buffer_km': buf_row['buffer_km']
            })
        print(f"{t_id} ({t_type}, {severity}): {', '.join(affected['COUNTRY'].values)}")
    else:
        print(f"{t_id} ({t_type}, {severity}): No land in damage zone (ocean impact)")
        damage_records.append({
            'country': 'Ocean (no land)',
            'threat_id': t_id,
            'threat_type': t_type,
            'severity': severity,
            'buffer_km': buf_row['buffer_km']
        })

# Create the summary table required by the assignment
damage_df = pd.DataFrame(damage_records)
print()
print('=== DAMAGE ZONE SUMMARY TABLE ===')
print(damage_df.to_string(index=False))

=== DAMAGE ZONE COUNTRY ANALYSIS ===

T001 (alien, High): Canada, United States
T002 (alien, High): No land in damage zone (ocean impact)
T003 (alien, High): Mexico
T004 (kaiju, Severe): No land in damage zone (ocean impact)
T005 (kaiju, Severe): Cuba, Dominican Republic, Haiti
T006 (alien, High): No land in damage zone (ocean impact)
T007 (alien, High): Mexico
T008 (airborne, Moderate): United States
T009 (alien, High): Canada
T010 (orbital, Critical): United States

=== DAMAGE ZONE SUMMARY TABLE ===
           country threat_id threat_type severity  buffer_km
            Canada      T001       alien     High        150
     United States      T001       alien     High        150
   Ocean (no land)      T002       alien     High        150
            Mexico      T003       alien     High        150
   Ocean (no land)      T004       kaiju   Severe        200
              Cuba      T005       kaiju   Severe        200
Dominican Republic      T005       kaiju   Severe        200
     

In [18]:
# ============================================================
# Aggregate: unique countries in damage zones
# ============================================================

severity_order = {'Critical': 4, 'Severe': 3, 'High': 2, 'Moderate': 1}

country_summary = []
for country in sorted(damage_df['country'].unique()):
    rows = damage_df[damage_df['country'] == country]
    worst = max(rows['severity'].values, key=lambda s: severity_order.get(s, 0))
    types = ', '.join(sorted(rows['threat_type'].unique()))
    threat_ids = ', '.join(sorted(rows['threat_id'].unique()))
    country_summary.append({
        'Country': country,
        'Threat IDs': threat_ids,
        'Threat Types': types,
        'Worst Severity': worst
    })

summary_df = pd.DataFrame(country_summary)
print('=== COUNTRY-LEVEL DAMAGE ASSESSMENT ===')
print(summary_df.to_string(index=False))
print()
print(f'Total countries in damage zones: {len(summary_df)}')

=== COUNTRY-LEVEL DAMAGE ASSESSMENT ===
           Country       Threat IDs             Threat Types Worst Severity
            Canada       T001, T009                    alien           High
              Cuba             T005                    kaiju         Severe
Dominican Republic             T005                    kaiju         Severe
             Haiti             T005                    kaiju         Severe
            Mexico       T003, T007                    alien           High
   Ocean (no land) T002, T004, T006             alien, kaiju         Severe
     United States T001, T008, T010 airborne, alien, orbital       Critical

Total countries in damage zones: 7


In [19]:
# ============================================================
# MILESTONE 5: Final comprehensive map with damage zones
# ============================================================

m5 = create_base_map(BASE_LAT, BASE_LON, zoom=3)

# World borders (base layer)
add_world_borders(m5, world)

# Highlight countries in damage zones
damage_countries = set(damage_df[damage_df['country'] != 'Ocean (no land)']['country'].values)
all_affected = all_intersected_countries.union(damage_countries)

affected_gdf = world_4326[world_4326['COUNTRY'].isin(all_affected)]
if len(affected_gdf) > 0:
    folium.GeoJson(
        affected_gdf.__geo_interface__,
        name='Affected Countries',
        style_function=lambda x: {
            'fillColor': '#E74C3C',
            'color': '#C0392B',
            'weight': 1.5,
            'fillOpacity': 0.35
        },
        tooltip=folium.GeoJsonTooltip(fields=['COUNTRY'], aliases=['Affected:'])
    ).add_to(m5)

# Add trajectories
add_trajectories(m5, threats, THREAT_STYLES)

# Add damage zone buffers (semi-transparent)
for idx, buf_row in buffers_4326.iterrows():
    buffer_geojson = mapping(buf_row.geometry)
    style = THREAT_STYLES[buf_row['type']]
    
    folium.GeoJson(
        buffer_geojson,
        name=f"{buf_row['id']} Damage Zone",
        style_function=lambda x, c=style['color']: {
            'fillColor': c,
            'color': c,
            'weight': 1.5,
            'fillOpacity': 0.2,
            'dashArray': '5 3'
        },
        tooltip=f"{buf_row['id']} damage zone ({buf_row['buffer_km']} km)"
    ).add_to(m5)

# Add threat origins
add_threat_origins(m5, threats, THREAT_STYLES)

# Add base marker and defense perimeter
add_base_marker(m5, BASE_LAT, BASE_LON, BASE_NAME)
folium.Circle(
    location=[BASE_LAT, BASE_LON],
    radius=BASE_PROXIMITY_KM * 1000,
    color='blue',
    fill=True,
    fillColor='blue',
    fillOpacity=0.05,
    weight=2,
    dash_array='10 5',
    tooltip=f'Base defense perimeter ({BASE_PROXIMITY_KM} km)'
).add_to(m5)

# Comprehensive legend
legend_html_m5 = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
            background-color: white; padding: 12px; border: 2px solid grey;
            border-radius: 5px; font-size: 12px; max-width: 220px;">
    <b>WDO Threat Analysis</b><br><br>
    <b>Trajectories:</b><br>
    <span style="color:#00FF00;">&#9644;</span> Alien (150 km zone)<br>
    <span style="color:#9400D3;">&#9644;</span> Orbital (300 km zone)<br>
    <span style="color:#FF8C00;">&#9644;</span> Airborne (100 km zone)<br>
    <span style="color:#FF0000;">- -</span> Kaiju (200 km zone)<br><br>
    <b>Map Layers:</b><br>
    <i style="background:#E74C3C;width:12px;height:12px;display:inline-block;"></i> Affected Country<br>
    <i style="background:rgba(100,100,100,0.2);width:12px;height:12px;display:inline-block;border:1px dashed;"></i> Damage Zone<br>
    <span style="color:blue;">- - -</span> Base Perimeter<br>
    <i style="background:#3388ff;width:12px;height:12px;display:inline-block;"></i> WDO Base
</div>
'''
m5.get_root().html.add_child(folium.Element(legend_html_m5))
folium.LayerControl().add_to(m5)

m5_path = MAPS_DIR / 'milestone_5_damage_zones.html'
m5.save(str(m5_path))
print(f'Milestone 5 map saved to: {m5_path}')

m5

Milestone 5 map saved to: /home/ubuntu/4545_Spatial_Data/Assignments/Project_01/maps/milestone_5_damage_zones.html


---
# Final Summary

## Complete Threat Assessment Report

This section consolidates all findings from the five milestones into a 
comprehensive WDO threat assessment.

In [20]:
# ============================================================
# FINAL SUMMARY: Complete threat assessment
# ============================================================

print('=' * 70)
print('       WORLD DEFENSE ORGANIZATION — THREAT ASSESSMENT REPORT')
print('=' * 70)
print(f'Base: {BASE_NAME}')
print(f'Base Coordinates: ({BASE_LAT}, {BASE_LON})')
print(f'Defense Perimeter: {BASE_PROXIMITY_KM} km')
print(f'Total Threats Analyzed: {len(threats)}')
print()

# Threat breakdown by type
type_counts = {}
for t in threats:
    type_counts[t["type"]] = type_counts.get(t["type"], 0) + 1
print('Threat Breakdown by Type:')
for ttype, count in sorted(type_counts.items()):
    print(f'  {ttype.capitalize():10s}: {count}')
print()

# Per-threat summary
print('-' * 70)
for t in threats:
    print(f"\n{t['id']} — {t['type'].upper()}")
    print(f"  Origin:          ({t['origin_lat']:.4f}, {t['origin_lon']:.4f})")
    print(f"  Destination:     ({t['dest_lat']:.4f}, {t['dest_lon']:.4f})")
    print(f"  Bearing:         {t['bearing_deg']} deg")
    print(f"  Speed:           {t['speed_kmh']:,.1f} km/h")
    print(f"  Duration:        {t['duration_min']:.0f} min")
    print(f"  Total Distance:  {t['total_distance_km']:,.1f} km")
    print(f"  Dist to Base:    {t['dist_to_base_km']:,.1f} km")
    print(f"  Closest to Base: {t['min_dist_to_base_km']:,.1f} km", end='')
    if t['passes_near_base']:
        print(' *** ALERT ***', end='')
    print()
    print(f"  Countries Crossed: {', '.join(t['intersected_countries']) if t['intersected_countries'] else 'None'}")
    dmg = t.get('damage_zone_countries', [])
    print(f"  Damage Zone:     {', '.join(dmg) if dmg else 'Ocean only'}")

print()
print('=' * 70)
print('                    END OF THREAT ASSESSMENT')
print('=' * 70)

       WORLD DEFENSE ORGANIZATION — THREAT ASSESSMENT REPORT
Base: WDO Command Center — Dallas, TX
Base Coordinates: (32.7767, -96.797)
Defense Perimeter: 500 km
Total Threats Analyzed: 10

Threat Breakdown by Type:
  Airborne  : 1
  Alien     : 6
  Kaiju     : 2
  Orbital   : 1

----------------------------------------------------------------------

T001 — ALIEN
  Origin:          (59.4707, -91.7663)
  Destination:     (48.5906, -92.2429)
  Bearing:         181.67 deg
  Speed:           978.6 km/h
  Duration:        74 min
  Total Distance:  1,210.2 km
  Dist to Base:    2,991.5 km
  Closest to Base: 1,802.2 km
  Countries Crossed: Canada
  Damage Zone:     Canada, United States

T002 — ALIEN
  Origin:          (10.0020, -114.1050)
  Destination:     (15.4682, -112.2080)
  Bearing:         18.5 deg
  Speed:           875.0 km/h
  Duration:        44 min
  Total Distance:  641.7 km
  Dist to Base:    3,091.8 km
  Closest to Base: 2,473.4 km
  Countries Crossed: None
  Damage Zone:     

In [21]:
# ============================================================
# Save the damage zone summary as CSV for Project 02 reuse
# ============================================================
csv_path = ROOT / 'data' / 'damage_zone_summary.csv'
damage_df.to_csv(csv_path, index=False)
print(f'Damage zone summary saved to: {csv_path}')

summary_csv_path = ROOT / 'data' / 'country_damage_assessment.csv'
summary_df.to_csv(summary_csv_path, index=False)
print(f'Country assessment saved to: {summary_csv_path}')

print()
print('All maps saved to:', MAPS_DIR)
print('Maps generated:')
for f in sorted(MAPS_DIR.glob('*.html')):
    print(f'  - {f.name}')

Damage zone summary saved to: /home/ubuntu/4545_Spatial_Data/Assignments/Project_01/data/damage_zone_summary.csv
Country assessment saved to: /home/ubuntu/4545_Spatial_Data/Assignments/Project_01/data/country_damage_assessment.csv

All maps saved to: /home/ubuntu/4545_Spatial_Data/Assignments/Project_01/maps
Maps generated:
  - milestone_1_world_map.html
  - milestone_2_threats.html
  - milestone_3_trajectories.html
  - milestone_4_intersections.html
  - milestone_5_damage_zones.html
